In [8]:
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_202"

In [9]:
os.environ["SPARK_HOME"] = "C:\HadoopEco\spark-3.2.1-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

In [3]:
import pandas as pd 
pd.set_option('max_colwidth',800)

### Create a spark context that includes a 3rd party jar for NLP

In [4]:
import pyspark

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [11]:
conf = pyspark.SparkConf().setAppName('nlp_analytics').setMaster('local')

In [12]:
sc = pyspark.SparkContext(conf=conf)

In [13]:
spark = SparkSession(sc)

In [ ]:
spark = SparkSession.builder \
    .appName("SentimentDetector")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jars", "/Users/maziyar/anaconda3/envs/spark/lib/python3.6/site-packages/sparknlp/lib/sparknlp.jar")\
    .config("spark.driver.extraClassPath", "/Users/maziyar/anaconda3/envs/spark/lib/python3.6/site-packages/sparknlp/lib/sparknlp.jar")\
    .config("spark.executor.extraClassPath", "/Users/maziyar/anaconda3/envs/spark/lib/python3.6/site-packages/sparknlp/lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [10]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

In [6]:
spark = SparkSession.builder\
    .config("spark.jars.packages","JohnSnowLabs:spark-nlp:1.8.2")\
    .getOrCreate()

RuntimeError: Java gateway process exited before sending its port number

### Read multiple files in a dir as one Dataframe

In [14]:
path = "./data/reddit-worldnews.json"
df = spark.read.json(path)

In [15]:
print(df.count())

100


In [16]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)
 |  

### Deal with Struct Type to query subfields

In [17]:
title = "data.title"
author = "data.author"
dfAuthorTitle = df.select(title, author)
dfAuthorTitle.limit(5).toPandas()

,title,author
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.",jaykirsch
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,canuck_burger
2,"Iranian ""morality police"" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.",honolulu_oahu_mod
3,"Trump administration 'pushing Saudi nuclear deal' which could benefit company linked to Jared Kushner - Senior Trump administration officials pushed a project to share nuclear power technology with Saudi Arabia over the objections of ethics officials, according to a congressional report",madam1
4,"NASA Happily Reports the Earth is Greener, With More Trees Than 20 Years Ago–and It's Thanks to China, India",purplexxx


### implement the equivalent of flatMap in dataframes

In [18]:
import pyspark.sql.functions as F

In [19]:
dfWordCount = df.select(F.explode(F.split(title,"\\s+")).alias("word"))\
    .groupBy("word").count().orderBy(F.desc("count"))
dfWordCount.limit(10).toPandas()

,word,count
0,to,58
1,the,46
2,of,42
3,in,41
4,a,25
5,for,20
6,and,19
7,from,12
8,on,11
9,with,10


### Use an NLP library to do Part-of-Speech Tagging

In [20]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp

In [36]:
dfAnnotated = bp.annotate(dfAuthorTitle, "title")

TypeError: 'JavaPackage' object is not callable